# Multiplication methods

### 1. Grid method

In the grid method, we split up the digits of each number and multiply them seperately, keeping track of the individual multiplications and the corresponding powers of ten. For this method we think of the numbers as strings, and perform single-digit multiplications using a table lookup. That is, we only multiply numbers using the "memorized" values of single digit multiplication, and thus our only computational task is to add up the products and distribute excesses to the correct powers of ten. (On paper we would use a grid to track the individual products, but this is not necessary in the algorithm.)

Example: Multiply 91 by 2.8

$91 \cdot 2.8$

= 

$ \ \ \ \ (9 \cdot 2)10^1 + (9 \cdot 8)10^0 $

$ + \ (1 \cdot 2)10^0 + (1 \cdot 8)10^{-1} $

=

$ (18)10^1 + (74)10^0 + (8)10^{-1} $

=

$ (25)10^1 + (4)10^0 + (8)10^{-1} $

=

$ (2)10^2 + (5)10^1 + (4)10^0 + (8)10^{-1} $

=

$254.8$

We use a custom class Rational to implement basic operations on decimal strings with fractional parts. The code for Rational.__mul__, which implements the grid method, is included in the commented block below.

In [1]:
from customnumbers import Rational


"""
def __mul__(self, other):
    #Multiply two Rationals by the grid method

    # 1. Compute products of individual digits, tracking powers of ten
    products = [
        (Rational.multTable[selfDigit][otherDigit], selfPower + otherPower)
        for selfDigit, selfPower in self.digitPowers
        for otherDigit, otherPower in other.digitPowers
    ]
    productsByPower = defaultdict(int)
    for product, power in products:
        productsByPower[power] += product

    # 2. Loop over components, adding runover to the next digit until
    #    all components are in the range 0-9
    lowestPower = min(productsByPower.keys())
    highestPower = max(productsByPower.keys())
    currentPower = lowestPower
    while currentPower <= highestPower:
        currentProduct = productsByPower[currentPower]
        if currentProduct > 9:
            productsByPower[currentPower] = currentProduct % 10
            productsByPower[currentPower+1] += currentProduct // 10
            if currentPower == highestPower:
                highestPower += 1
        currentPower += 1

    # 3. Construct and return the Rational from the components
    components = list(productsByPower.items())
    components.sort(reverse=True, key=itemgetter(0))
    digits = [c[1] for c in components]
    hp = components[0][0]
    isNegative = (self.isNegative and not other.isNegative) or (not self.isNegative and other.isNegative)
    return Rational.fromDigits(digits, highestPower=hp, negative=isNegative)
"""

        
def gridMultiply(a, b):
    return (Rational(str(a))*Rational(str(b))).asFloat()

def compareMethods(a, b, multFn, fnName):
    """Print comparison of multFn and native multiplication"""
    
    fnProduct = multFn(a, b)
    nativeProduct = a*b
    print('Product of ' + str(a) + ' and ' + str(b))
    print('Native multiplication: ' + str(nativeProduct))
    print(fnName + ': ' + str(fnProduct))
    print('----')
    
def testMethod(a, b, multFn, fnName=None, pctError=10**(-8), verbose=False):
    """Test if multFn, native multiplication produce same result"""
    
    fnProduct = multFn(a, b)
    nativeProduct = a*b
    if verbose:
        print('Testing product of ' + str(a) + ' and ' + str(b))
        print('Native multiplication: {}'.format(nativeProduct))
        if fnName:
            print('{}: {}'.format(fnName, fnProduct))
        else:
            print('Alternate multiplication: {}'.format(fnProduct))
        print('----')
    return abs(nativeProduct - fnProduct)/nativeProduct < pctError

def runTests(testCases, multFn, fnName=None, pctError=10**(-8), verbose=False):
    """Run multiplication tests; return list of bools indicating tests passed"""
    
    testResults = [
        testMethod(a, b, multFn, fnName, pctError, verbose)
        for a, b in testCases
    ]
    if verbose:
        passed = sum(testResults)
        total = len(testResults)
        print('\nTest results:')
        print('Passed {} out of {} cases'.format(passed, total))
    return testResults

As = [12.56, 1.466, .09484, 48, 8]
Bs = [9.89, 245.256, .9834, 98.20001, 43]
testCases =  list(zip(As, Bs))
testCases += list(zip(As, reversed(Bs)))
testCases += list(zip(As, [-1*x for x in Bs]))
testCases += list(zip(reversed(As), [-1*x for x in Bs]))

testResults = runTests(testCases, multFn=gridMultiply, fnName='Grid method', verbose=True)

Testing product of 12.56 and 9.89
Native multiplication: 124.21840000000002
Grid method: 124.2184
----
Testing product of 1.466 and 245.256
Native multiplication: 359.545296
Grid method: 359.545296
----
Testing product of 0.09484 and 0.9834
Native multiplication: 0.093265656
Grid method: 0.093265656
----
Testing product of 48 and 98.20001
Native multiplication: 4713.60048
Grid method: 4713.60048
----
Testing product of 8 and 43
Native multiplication: 344
Grid method: 344.0
----
Testing product of 12.56 and 43
Native multiplication: 540.08
Grid method: 540.08
----
Testing product of 1.466 and 98.20001
Native multiplication: 143.96121466
Grid method: 143.96121466
----
Testing product of 0.09484 and 0.9834
Native multiplication: 0.093265656
Grid method: 0.093265656
----
Testing product of 48 and 245.256
Native multiplication: 11772.288
Grid method: 11772.288
----
Testing product of 8 and 9.89
Native multiplication: 79.12
Grid method: 79.12
----
Testing product of 12.56 and -9.89
Native mu

### 2. Quarter square multiplication

Quarter square multiplication is based on the following interesting algebraic identity:
    
$ xy = \frac{1}{4}(4xy) = \frac{1}{4}((x^2 + 2xy + y^2) - (x^2 - 2xy + y^2)) = \frac{(x + y)^2}{4} - \frac{(x - y)^2}{4} $

In fact,  when x and y are integers, we can replace $ \frac{(x + y)^2}{4} - \frac{(x - y)^2}{4} $ with
$ \frac{(x + y)^2}{4} - \frac{(x - y)^2}{4} $ because: if x+y, x-y are both even, then 4 divides $(x+y)^2$ and $(x-y)^2$ so the floor operator does nothing; and if x+y, x-y are both odd, then the remainders after dividing by 4 will cancel out due to the subtraction.

So, to simplify things, we will restrict to multiplying positive integers. Thanks to the identity, we can compute the product by doing a table lookup of (floors of) quarter squares. Such tables used to be published for this purpose, but we can generate our own table of quarter squares from 1 to 200,000, allowing us to multiply numbers up to 100,000 X 100,000.

In [2]:
class QSMultiplier():

    quarterSquareTable = {x: x**2 // 4 for x in range(200000)}
    
    def __init__(self):
        pass
    
    def multiply(self, x, y):
        a = x + y
        b = abs(x - y)
        return QSMultiplier.quarterSquareTable[a] - QSMultiplier.quarterSquareTable[b]


def QSMultiply(a, b):
    return QSMultiplier().multiply(a, b)

As = [43376, 41687, 55303, 6207, 29420, 19]
Bs = [12158, 58554, 342, 63819, 39481, 37334]
testFactors = list(zip(As, Bs)) + list(zip(As, reversed(Bs)))

testResults = runTests(testFactors, multFn=QSMultiply, fnName='Quarter square method', verbose=True)

Testing product of 43376 and 12158
Native multiplication: 527365408
Quarter square method: 527365408
----
Testing product of 41687 and 58554
Native multiplication: 2440940598
Quarter square method: 2440940598
----
Testing product of 55303 and 342
Native multiplication: 18913626
Quarter square method: 18913626
----
Testing product of 6207 and 63819
Native multiplication: 396124533
Quarter square method: 396124533
----
Testing product of 29420 and 39481
Native multiplication: 1161531020
Quarter square method: 1161531020
----
Testing product of 19 and 37334
Native multiplication: 709346
Quarter square method: 709346
----
Testing product of 43376 and 37334
Native multiplication: 1619399584
Quarter square method: 1619399584
----
Testing product of 41687 and 39481
Native multiplication: 1645844447
Quarter square method: 1645844447
----
Testing product of 55303 and 63819
Native multiplication: 3529382157
Quarter square method: 3529382157
----
Testing product of 6207 and 342
Native multiplicat

### 3. Karatsuba multiplication

If you had to multiply two numbers by hand, you would probably write both numbers down in rows, multiplying the digit pairs in your head and writing down the results, tallying overflows, and adding up the results:

<pre>
    234510  
    x  998  
    ------
   1876080
  21105900
 211059000
 ---------
 234040980
</pre>

This is called long multiplication, or the standard algorithm. A weakness of this method is that if we have two n-digit numbers, then the time to multiply them by long multiplication is $O(n^2)$; i.e., the number of operations required scales at about the rate of the square of the size of the numbers. For extremely large numbers this can become a problem.

Karatsuba multiplication is a faster algorithm that requires $O(n^{\log_2 3}) \approx O(n^{1.585})$ operations. It achieves this speed increase by dividing the multiplication of two n-digit numbers into 3 multiplications involving numbers with less than n digits; then repeating that step on the three new multiplications, dividing them into multiplications involving even fewer digits, and so on, until finally we are only multiplying single-digit numbers. Because each step requires more addition/substraction operations than long multiplication, long multiplication is actually faster for small inputs. But for very large numbers (e.g. 1000 digits or more), the greatly reduced number of multiplications required ($n^{1.585}$ instead of $n^2$) makes Karatsuba multiplication much faster.

So how does the base step work? Suppose x and y are numbers specified as decimal strings of length n. Then let $m = \lceil \frac{n}{2} \rceil$. We can write $x = x_1 10^m + x_0$ and $y = y_1 10^m + y_0$, where $x_0$ and $y_0$ are less than $10^m$. Then the product can be written

$ xy = (x_1 10^m + x_0)(y_1 10^m + y_0) = (x_1 y_1) 10^{2m} + (x_0 y_1 + x_1 y_0) 10^{m} + x_0 y_0 $

To compute the right-hand side as it's written above, we still need to do 4 multiplications involving $x_0, x_1, y_0, y_1$. However we can reduce this to 3 by writing the middle coefficient in terms of the other two:

$x_0 y_1 + x_1 y_0 = (x_0 - x_1)(y_1 - y_0) + x_1 y_1 + x_0 y_0$

Now, we only have to do 3 multiplications involving shorter decimal strings, some additions/subtractions, and multiplications by powers of 10 (merely adding digits to the decimal strings). We can recursively compute the 3 multiplications, each time halving the lengths of the numbers being multiplied, resulting in a total number of multiplications of at most $n^{\log_2 3}$.

So the procedure consists of the following steps:
1. Write x, y as $x_1 10^m + x_0$ and $y_1 10^m + y_0$, such that $x_0, x_1, y_0, y_1$ are at most $m$ digits.
2. Recursively compute the products $x_0 y_0$, $x_1 y_1$, and $(x_0 - x_1)(y_1 - y_0)$.
3. Use these values and some addition/subtraction operations to compute the middle coefficient $x_0 y_1 + x_1 y_0$
4. Finally use addition and multiplication by $10^m$ to compute $ (x_1 y_1) 10^{2m} + (x_0 y_1 + x_1 y_0) 10^{m} + x_0 y_0 = xy. $

We use a custom Number class to handle basic operations with decimal strings, so we can simply implement the Karatsuba multiplication algorithm below.

In [3]:
from customnumbers import Number
from math import ceil

class Karatsuba():
    
    def __init__(self):
        pass
    
    def multiply(self, xString, yString):
        """
        Multiply decimal strings by Karatsuba multiplication
        
        Args: 
        -- xString, yString: decimal strings (in usual order) of positive integers to multiply
        
        Returns: decimal string (in usual order) of the product
        """
        
        x = Number(xString)
        y = Number(yString)
        return str(self._multiply(x, y))
    
    def _multiply(self, x, y):
        """Recursively multiply Numbers x, y (implement Karatsuba algorithm)"""
        
        Lx, Ly = len(x), len(y)
        
        # Base of induction; called once numbers are reduced to single digits
        if Lx < 2 and Ly < 2:
            return Number.multiplySingleDigits(x, y)
        
        # Split off 10**m where m = ceil( max(len(x),len(y)) / 2 )
        m = ceil(max(Lx, Ly)/2)
        x0, x1 = x.decimalDecompose(m)
        y0, y1 = y.decimalDecompose(m)
        
        # Recursively compute three coeffs
        z1 = self._multiply(x1, y1)
        z3 = self._multiply(x0, y0)
        z2 = self._multiply(x0 - x1, y1 - y0) + z1 + z3
        
        # Use decimal string addition/ten multiplication to compute and return xy
        z1.multiplyTenPower(2*m)
        z2.multiplyTenPower(m)
        return z1 + z2 + z3

    
def KaratsubaMultiply(a, b):
    return int(Karatsuba().multiply(str(a), str(b)))

As = [43376, 41687, 55303, 6207, 29420, 19]
Bs = [12158, 58554, 342, 63819, 39481, 37334]
testFactors = list(zip(As, Bs)) + list(zip(As, reversed(Bs)))

testResults = runTests(testFactors, multFn=KaratsubaMultiply, fnName='Karatsuba algorithm', verbose=True)

Testing product of 43376 and 12158
Native multiplication: 527365408
Karatsuba algorithm: 527365408
----
Testing product of 41687 and 58554
Native multiplication: 2440940598
Karatsuba algorithm: 2440940598
----
Testing product of 55303 and 342
Native multiplication: 18913626
Karatsuba algorithm: 18913626
----
Testing product of 6207 and 63819
Native multiplication: 396124533
Karatsuba algorithm: 396124533
----
Testing product of 29420 and 39481
Native multiplication: 1161531020
Karatsuba algorithm: 1161531020
----
Testing product of 19 and 37334
Native multiplication: 709346
Karatsuba algorithm: 709346
----
Testing product of 43376 and 37334
Native multiplication: 1619399584
Karatsuba algorithm: 1619399584
----
Testing product of 41687 and 39481
Native multiplication: 1645844447
Karatsuba algorithm: 1645844447
----
Testing product of 55303 and 63819
Native multiplication: 3529382157
Karatsuba algorithm: 3529382157
----
Testing product of 6207 and 342
Native multiplication: 2122794
Karat

Of course the whole point of the Karatsuba algorithm is to efficiently multiply very large numbers, so let's compare the speed of our different multiplication methods on large numbers.

In [19]:
from math import floor
import random


def nDigitRandomNumber(nDigits):
    return int(''.join([str(floor(random.random()*10)) for _ in range(nDigits)]))

def timeMultMethods(a, b):
    """Compare run times of multiplication methods computing a*b"""
    
    ra = Rational(str(a))
    rb = Rational(str(b))

    na = Number(str(a))
    nb = Number(str(b))

    K = Karatsuba()

    print('Native multiplication:')
    %time str(a*b)

    print('----')
    print('Grid multiplication:')
    %time str(ra*rb)


    print('----')
    print('Karatsuba multiplication:')
    %time K.multiply(str(a), str(b))
    

# Comparing time for different digit lengths
for nDigits in [100, 1000, 5000, 10000]:
    print('\nTesting numbers of length {}\n--------\n'.format(nDigits))
    timeMultMethods(nDigitRandomNumber(nDigits), nDigitRandomNumber(nDigits))


Testing numbers of length 100
--------

Native multiplication:
Wall time: 1 ms
----
Grid multiplication:
Wall time: 106 ms
----
Karatsuba multiplication:
Wall time: 172 ms

Testing numbers of length 1000
--------

Native multiplication:
Wall time: 1 ms
----
Grid multiplication:
Wall time: 2.05 s
----
Karatsuba multiplication:
Wall time: 3.86 s

Testing numbers of length 5000
--------

Native multiplication:
Wall time: 3 ms
----
Grid multiplication:
Wall time: 1min 6s
----
Karatsuba multiplication:
Wall time: 1min 9s

Testing numbers of length 10000
--------

Native multiplication:
Wall time: 10 ms
----
Grid multiplication:
Wall time: 17min 37s
----
Karatsuba multiplication:
Wall time: 5min 8s


### 4. Fast Fourier Transform method

#### Polynomial multiplication and convolution - why we use the Fourier transform

When we multiply two base-10 numbers, we're essentially multiplying polynomials. For example:

$ 105893 \cdot 3819 = (3 \cdot 10^0 + 9 \cdot 10^1 + 8 \cdot 10^2 + 5 \cdot 10^3 + 0 \cdot 10^4 + 1 \cdot 10^5) 
\cdot (9 \cdot 10^0 + 1 \cdot 10^1 8  + \cdot 10^2 + 3 \cdot 10^3) $

To get the coefficients of the product, we have to take the two series of coefficients of the the original polynomials and compute their convolution. If the original coefficients are $a = \{a_0, a_1, ... , a_{N-1}\}$ and $b = \{b_0, b_1, ... , b_{N-1}\}$, then the convolution is the series $c = a \ast b = \{c_0, c_1, ... , c_{N-1}\}$ where $c_k$ is defined as:

$ c_k = \sum_{n=0}^{n=k} a_{n} \cdot b_{k-n} $

For example: 

$c_0 = a_0 b_0$, 

$c_1 = a_0b_1 + a_1b_0$,

$c_2 = a_0b_2 + a_1b_1 + a_2b_0$,

and so on. This requires computing about $N^2$ products in total. However, we can actually reduce the number of products we need to compute to just $N$ by applying the Discrete Fourier Transform (DFT). The DFT maps our original series $a$ and $b$ to new series $\mathcal{F}(a)$, $\mathcal{F}(b)$. The exact definition isn't important right now; the key property is that the DFT converts convolutions into products:

$\mathcal{F}(a \ast b)_k = \mathcal{F}(a)_k \cdot \mathcal{F}(b)_k$

Hence, instead of computing the convolution of $a$ and $b$ (requiring $N^2$ products) we can transform $a$ and $b$ into $\mathcal{F}(a)$ and $\mathcal{F}(b)$, compute the elementwise products of $\mathcal{F}(a)$ and $\mathcal{F}(b)$ (just $N$ products), and then invert the transformation to recover the coefficients $c$. As long as we can quickly compute the DFT $\mathcal{F}$ (specifically, faster than O(N^2)), this will be faster than computing the convolution directly.

#### Cooley-Tukey algorithm - a fast Fourier transform algorithm

Given a series $x = \{x_0, ... , x_{N-1}\}$, the Discrete Fourier Transform is the series $X = \mathcal{F}(x)$ where $X_k$ is defined as:

$ X_k = \sum_{n=0}^{N-1}x_n \cdot e^{- \frac{2 \pi i}{N} n k} $

In general this requires computing about $N^2$ products of complex numbers - which is slow. Instead we will use a Fast Fourier Transform (FFT) algorithm called the Cooley-Tukey algorithm, which is $O(N \log N)$ and only requires that $N$ be a power of 2 - which is not a problem for us, since we can simply pad the series with zeros until its length is a power of 2.

The fundamental idea of the Cooley-Tukey algorithm is that we can compute the DFT of $x$ by computing, separately, the DFT of the sequence of even-indexed entries of $x$ and the sequence of odd-indexed entries of $x$. These sequences have half the length of the original sequence $x$. Hence, we convert the computation of a DFT on a sequence of length $N$ into two computations of DFTs on sequences of length $\frac{N}{2}$. Since $N$ is a power of 2, we can repeat this recursively to end up with a computation of complexity $O(N \log N)$. 

But in order to make this work, we must be able to combine the transformed even- and odd-indexed series back to the transform of the original series. This relies on a little bit of algebra:

\begin{align*}
    X_k &= \sum_{n=0}^{N-1}x_n \cdot e^{- \frac{2 \pi i}{N} n k} \\
        &= \sum_{m=0}^{N/2 - 1} x_{2m} e^{- \frac{2 \pi i}{N} (2m) k} \ + \ 
                \sum_{m=0}^{N/2 - 1} x_{2m + 1} e^{- \frac{2 \pi i}{N} (2m + 1) k} \\
        &= \sum_{m=0}^{N/2 - 1} x_{2m} e^{- \frac{2 \pi i}{N/2} m k} \ + \ 
                (e^{- \frac{2 \pi i}{N} k}) \sum_{m=0}^{N/2 - 1} x_{2m + 1} e^{- \frac{2 \pi i}{N} m k} \\
        &= E_k + e^{- \frac{2 \pi i}{N} k} \cdot O_k
\end{align*}

First, we split the definition of $X_k$ into sums involving the even- and odd-indexed subsequences; then, we rearrange the exponents and remove a constant factor from the odd-indexed sum; finally, we observe that the resulting sums are just the entries $E_k$, $O_k$ of the transformed even- and odd-indexed sequences.

A second calculation allows us to compute $X_{k + \frac{N}{2}}$ from $E_k$ and $O_k$:

\begin{align*}
    X_{k+\frac{N}{2}} &= E_{k+\frac{N}{2}} + e^{- \frac{2 \pi i}{N} (k+\frac{N}{2})} \cdot O_{k+\frac{N}{2}} \\
        &= \sum_{m=0}^{N/2 - 1} x_{2m} e^{- \frac{2 \pi i}{N} (2m) (k+\frac{N}{2})} \ + \ 
                e^{- \frac{2 \pi i}{N} (k+\frac{N}{2})} \cdot 
                    \sum_{m=0}^{N/2 - 1} x_{2m + 1} e^{- \frac{2 \pi i}{N} (2m + 1) (k+\frac{N}{2})} \\
        &= ......
\end{align*}


#### The big picture